In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
prompt = "It was a dark and stormy"
print(prompt)

It was a dark and stormy


In [3]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
input_ids = tokenizer(prompt).input_ids

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [4]:
input_ids

[2132, 572, 264, 6319, 323, 13458, 88]

In [5]:
for t in input_ids:
  print(t,"\t:", tokenizer.decode(t))

2132 	: It
572 	:  was
264 	:  a
6319 	:  dark
323 	:  and
13458 	:  storm
88 	: y


In [6]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [7]:
#return pyTorch tensors
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

In [8]:
output = model(input_ids)
output.logits.shape

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


torch.Size([1, 7, 151936])

In [9]:
#predict the next word of the sequence
final_logits = model(input_ids).logits[0,-1]
final_logits.argmax()


tensor(3729)

In [10]:
tokenizer.decode(final_logits.argmax())

' night'

In [11]:
#top 10 potential next tokens
import torch
top10_logits = torch.topk(final_logits, 10)
for index in top10_logits.indices:
  print(tokenizer.decode(index))


 night
 evening
 day
 morning
 winter
 afternoon
 Saturday
 Sunday
 Friday
 October


In [12]:
#find how confident the model is about each prediction
top10 = torch.topk(final_logits.softmax(dim=0), 10)


In [13]:
for value, index in zip(top10.values, top10.indices):
  print(f"{tokenizer.decode(index):<10}{value.item():.2%}")

 night    88.71%
 evening  4.30%
 day      2.19%
 morning  0.49%
 winter   0.45%
 afternoon0.27%
 Saturday 0.25%
 Sunday   0.19%
 Friday   0.17%
 October  0.16%


## Generating Text

In [14]:
output_ids = model.generate(input_ids,max_new_tokens = 20)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [15]:
decoded_text = tokenizer.decode(output_ids[0])
print(decoded_text)

It was a dark and stormy night. The sky was dark and the wind was howling. The rain was pouring down and the


In [16]:
print("Input IDs",input_ids[0])
print("Output IDs",output_ids)
print(f"Generated text:{decoded_text}")

Input IDs tensor([ 2132,   572,   264,  6319,   323, 13458,    88])
Output IDs tensor([[ 2132,   572,   264,  6319,   323, 13458,    88,  3729,    13,   576,
         12884,   572,  6319,   323,   279,  9956,   572,  1246,  2718,    13,
           576, 11174,   572, 50413,  1495,   323,   279]])
Generated text:It was a dark and stormy night. The sky was dark and the wind was howling. The rain was pouring down and the


In [17]:
beam_ouptut = model.generate(input_ids, num_beams = 5, max_new_tokens =30)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [18]:
print(tokenizer.decode(beam_ouptut[0]))

It was a dark and stormy night. The wind was howling, and the rain was pouring down. The sky was dark and gloomy, and the air was filled with the


In [19]:
#reptition penalty
beam_ouput = model.generate(input_ids,num_beams = 5,repetition_penalty = 2.0, max_new_tokens = 38)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [20]:
#sampling
from transformers import set_seed
set_seed(70)

In [21]:
sampling_ouptut = model.generate(input_ids,do_sample = True, max_new_tokens = 34,
                                 top_k =0)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [22]:
print(tokenizer.decode(sampling_ouptut[0]))

It was a dark and stormy night. Kevin said he was going to stay up all night, staring at the cloudless stars, wondering, what if I lost my dream.He’d been teasing her about


In [23]:
sampling_output = model.generate(input_ids, do_sample = True, temperature = 0.4, max_new_tokens = 40,
                                 top_k = 0)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [24]:
print(tokenizer.decode(sampling_output[0]))

It was a dark and stormy night. The sky was dark and the wind howled. The rain wasURNING and the thunder was thundering. The thunder was so loud that the entire town could hear\xbseveryone was


In [25]:
sampling_output = model.generate(input_ids, do_sample = True, temperature = 0.001, max_new_tokens = 40,
                                 top_k = 0)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [26]:
print(tokenizer.decode(sampling_output[0]))

It was a dark and stormy night. The sky was dark and the wind was howling. The rain was pouring down and the lightning was flashing. The sky was dark and the wind was howling. The rain was pouring down


In [27]:
sampling_output = model.generate(input_ids, do_sample=True, max_new_tokens=40,
                                 top_p = 0.94,top_k=10)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [28]:
print(tokenizer.decode(sampling_ouptut[0]))

It was a dark and stormy night. Kevin said he was going to stay up all night, staring at the cloudless stars, wondering, what if I lost my dream.He’d been teasing her about


## Zero shot generation


In [29]:
#zero shot generation  - predicts without training data
tokenizer.encode(" positive"), tokenizer.encode(" negative")

([6785], [8225])

In [30]:
#run inference with the model
def score(review):
  prompt = f""" Question: Is the review positive or negative about the movie?
  Review: {review}
  Answer: """
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  final_logits = model(input_ids).logits[0,-1]

  if final_logits[6785] > final_logits[8225]:
    print("Positive")
  else:
    print("Negative")

In [31]:
score("I loved the movie")

Positive


In [32]:
score("The movie was terrible")

Negative


In [33]:
score("A complex yet wonderful film about the depravity of man")

Positive


# Few Shot generalization


In [34]:
prompt = """\
Translate English to Spanish:

English: I do not speak Spanish.
Spanish: No hablo español.

English: See you later!
Spanish: ¡Hasta luego!

English: Where is a good restaurant?
Spanish: ¿Dónde hay un buen restaurante?

English: What rooms do you have available?
Spanish: ¿Qué habitaciones tiene disponibles?

English: I like soccer
Spanish:"""
inputs = tokenizer(prompt, return_tensors="pt").input_ids
output = model.generate(
    inputs,
    max_new_tokens=10,
)

print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Translate English to Spanish:

English: I do not speak Spanish.
Spanish: No hablo español.

English: See you later!
Spanish: ¡Hasta luego!

English: Where is a good restaurant?
Spanish: ¿Dónde hay un buen restaurante?

English: What rooms do you have available?
Spanish: ¿Qué habitaciones tiene disponibles?

English: I like soccer
Spanish: Me gusta el fútbol

English:
